# YOLOv8 Model for Medical Masks Detection
Implementation of Yolov8 model trained using dataset created using three different datasets. Model will be implemented in DeepStream SDK Face Mask Absence Detection.
Process: 
1. Data processing
2. Training Yolov8 custom model
3. Converting it to ONNX file (Open Neural Network Exchange)
4. Modifying model using GraphSurgeon for DeepStream compatibility
4. Transferring it to TensorRT engine (compatibility, performance, optimization)
5. Creating custom C++ parser for the Deepstream application
6. Programming Deepstream Application and creating configuration file
7. Analyzing results and performance

**Environment and GPU Check**

In [1]:
import os
import re
import time
import yaml
import shutil

import torch
import torch.onnx
from ultralytics.models import YOLO

#import wandb
from roboflow import Roboflow
#from wandb.integration.ultralytics import add_wandb_callback


!nvidia-smi
print("Cuda available") if torch.cuda.is_available() else print("Cuda is not available")
HOME = 'C:/Users/kacpi/PycharmProjects/Medical-Mask-Presence-Detection'

Mon Nov 20 17:52:57 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...    On  | 00000000:01:00.0 Off |                  N/A |
| N/A   38C    P0              N/A /  80W |      9MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#wandb.login()

**Parameters**
* Project and version are needed for Roboflow deployment where model can be used and analyzed
* Epochs and Bs are both used in training process
* Img_size = 640 

In [5]:
PROJECT = "Medical-Mask-Presence-Detection"
VERSION = 14
EPOCHS = 40
BS = 64
IMG_SIZE = 640

**Dataset Preparation**
1. downloading dataset from Roboflow platform
2. checking if dataset already exists in local environment

In [6]:
version_name = f'Mask-Detection-YOLOv8-{VERSION}'
full_dataset_path = os.path.join(HOME, 'YOLO', version_name)

if os.path.exists(full_dataset_path):
    print(f"{version_name} already exists in the directory!")
    dataset_location = full_dataset_path
else:
    # Download dataset from Roboflow
    with open('apiKey.txt') as api:
        rf = Roboflow(api_key=api.read())
        project = rf.workspace("agh-ett2f").project("mask-detection-yolov8")
        dataset = project.version(VERSION).download("yolov8")
        dataset_location = dataset.location

    shutil.copy('Mask-Detection-YOLOv8-1/data_backup.yaml', f'Mask-Detection-YOLOv8-{VERSION}/data.yaml')

print("Dataset location:", dataset_location)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.208, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Mask-Detection-YOLOv8-14 in yolov8:: 100%|██████████| 9106/9106 [00:00<00:00, 12122.04it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'Mask-Detection-YOLOv8-1/data_backup.yaml'

**Training the Model**
* Weights and Biases initialization
* starting a timer to count a time it took for model to train
* train init with hiperparameters to set for fine-tuning process

In [ ]:
# Initialize the YOLOv8 model
model = YOLO(f"yolov8s.pt")
save_dir_train = f'E:/PycharmProjects/Medical-Mask-Presence-Detection/runs/detect/train{VERSION}'
os.makedirs(save_dir_train, exist_ok=True)

# Set up Weights and Biases for experiment tracking
wandb.init(project=PROJECT, config={"epochs": EPOCHS, "batch_size": BS})
add_wandb_callback(model, enable_model_checkpointing=True)

# Training the model
start_time = time.time()

model.train(
    project=PROJECT,
    data=f'{dataset_location}/data.yaml',
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    save_dir=save_dir_train,
    seed=3,
    lr0=0.005,
    warmup_epochs=3,
    lrf=0.0005,
    weight_decay=0.0003,
    dropout=0.2,
)

end_time = time.time() - start_time

print(f"Training took {end_time/60:.2f} minutes ({end_time:.2f} seconds)")

**Validation and Testing**
1. defining func that will find lates version of model weights
2. defining path used in model validation and evaluation

In [ ]:
# Helper function to retrieve the directory with the latest training
def train_dir_with_biggest_number():
    directory = f'{HOME}/YOLO/Medical-Mask-Presence-Detection/'
    folders = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    pattern = r'^train(\d+)$'
    max_num = -1
    max_folder = None

    for folder in folders:
        match = re.match(pattern, folder)
        if match:
            num = int(match.group(1))
            if num > max_num:
                max_num = num
                max_folder = folder
        else:
            max_folder = "train"

    return max_folder

max_train = train_dir_with_biggest_number()
print("Saved to: " + str(max_train))

trainedModelPath = f'{HOME}/YOLO/Medical-Mask-Presence-Detection/{max_train}/weights/best.pt'
trainedModelDir=f'Medical-Mask-Presence-Detection/train/'
trainedModel = YOLO(trainedModelPath)

**Validating and predicting model**

In [ ]:
# # Validate the model
trainedModel.val(data=f'{dataset_location}/data.yaml')

# Predict using the trained model on test images
trainedModel.predict(conf=0.25, source=f'{dataset.location}/test/images')

Deployment to Roboflow platform

In [ ]:
# Deploying the model for inference
project.version(VERSION).deploy(model_type='yolov8', model_path=trainedModelDir)

In [ ]:
# Clear GPU memory for optimization (import to run only one cell if needed)
import torch
torch.cuda.empty_cache()